In [233]:
source("tools.R")

Parsed with column specification:
cols(
  .default = col_double(),
  userId = col_character(),
  age = col_character(),
  datetime = col_character(),
  gender = col_character(),
  datetime_1 = col_character(),
  stage = col_integer(),
  scenario = col_character(),
  subcondition = col_integer(),
  pageIndex = col_integer(),
  noiseIndex = col_integer(),
  day0 = col_character(),
  day1 = col_character(),
  day2 = col_character(),
  day3 = col_character(),
  day4 = col_character(),
  day5 = col_character(),
  day6 = col_character(),
  day7 = col_character(),
  day8 = col_character(),
  day9 = col_character()
  # ... with 69 more columns
)
See spec(...) for full column specifications.
Warning message in eval(substitute(expr), envir, enclos):
"NAs introduced by coercion"

In [234]:
library(gridExtra)

In [235]:
# Kernel composition name change
#l+r+p -> l+p+r
#l+r*p -> l+p*r
#l*r*p -> l*p*r

readable_kernel <- function(current_kernel){
    new_kernel <- current_kernel
    
    new_kernel <- ifelse(new_kernel == 'l+r+p', 'l+p+r', new_kernel)
    new_kernel <- ifelse(new_kernel == 'l+r*p', 'l+p*r', new_kernel)
    new_kernel <- ifelse(new_kernel == 'l*r*p', 'l*p*r', new_kernel)
    
    return(new_kernel)
}

# Plotting functions

In [236]:
plot_lmls <- function(prop_data, title='', scenario, hide_x=FALSE, hide_y=FALSE) {
    
    # Best-fitting real-world data kernel composition
    rw_lmls <- read_csv("data/real-world/to-plot.csv")
    red_kernel <- (rw_lmls %>%
                    filter(scenario == !!scenario) %>%
                    summarize(red_kernel = kernel[which.max(lml)]))$red_kernel
    red_kernel <- readable_kernel(red_kernel)

    plot <- prop_data %>%
                ggplot(aes(x=kernel, y=value)) +
                    geom_bar(stat='identity') +
                    geom_bar(stat='identity', data=prop_data %>% filter(kernel == red_kernel), alpha=0, size=0.5, color="red") +
                    coord_cartesian (ylim=c(0, 1)) +
                    #labs(title = title) +
                    ggthemes::theme_few() +
                    xlab("Kernel composition") +
                    ylab("Mean likelihood (scaled)") +
                    geom_errorbar(aes(ymin=as.numeric(lo_ci), ymax=as.numeric(hi_ci)),
                                      width=.3,
                                      position=position_dodge(.9)) +
                    scale_y_continuous(breaks = seq(0, 1, length.out=3)) +
                    theme(axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          text = element_text(size=12, family="serif"))

    if(hide_x){
        plot <- plot + theme(axis.title.x=element_blank())
    }
    if(hide_y){
        plot <- plot + theme(axis.title.y=element_blank())
    }
    
    return(plot)
}

In [237]:
plot_curves <- function(plot_data, plot_rwdata, ylab='', title='', hide_x=FALSE, hide_y=FALSE) {

    # Min and max value of the data to be shown
    min_y <- signif(min(plot_data$value), digits=1)
    max_y <- signif(max(plot_data$value), digits=1)
    
    if(abs(min_y) < 5){
        min_y = 0
    } else if(abs(min_y) < 10) {
        if(sign(min_y) == -1){
            min_y = -10
        } else {
            min_y = 0
        }
    }
    
    # Mean trend
    trend <- plot_data %>% group_by(day) %>% summarize(mean_y = mean(value))
    
    # Plotting
    plot <- plot_data %>%
                ggplot(aes(x=day, y=value, group=id)) +
                    geom_line(col="steelblue", alpha=0.2) + # Participants' curves
                    geom_line(data=plot_rwdata, aes(x=day, group=1), colour="red", alpha=0.6) + # Real world data
                    geom_line(data=trend, aes(x=day, y=mean_y, group=1), colour="black") + # Mean trend
                    #labs(title = title) +
                    ggthemes::theme_few() +
                    ylab(ylab) +
                    scale_x_continuous(breaks = c(0, 365, 365*2, 365*3), labels=c('Y1', 'Y2', 'Y3', 'Y4')) +
                    scale_y_continuous(breaks = seq(min_y, max_y, length.out=3)) +
                    theme(axis.text.x = element_text(angle = 90,  vjust = 0.5, hjust=0),
                          axis.text.y = element_text(angle = 90,  vjust = 0.5, hjust=0.5),
                          text = element_text(size=12, family="serif"))

    if(hide_x){
        plot <- plot + theme(axis.title.x=element_blank())
    }
    if(hide_y){
        plot <- plot + theme(axis.title.y=element_blank())
    }
    
    return(plot)
}

# Prior: Kernel compositions bar plots

#### Import and prepare the data

In [238]:
data_prior <- read_csv("data/for_composititional_analysis_prior.csv")

dict_prior <- data_prior %>%                        
                        group_by(id, pid, scenario) %>%
                        summarize()

lmls_prior <- read_csv("output/lmls_prior.csv")

metadata_prior <- read_csv("output/metadata_prior.csv")

Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_integer(),
  id = col_integer(),
  pid = col_character(),
  scenario = col_character(),
  x = col_integer(),
  y = col_double()
)
Warning message:
"Missing column names filled in: 'X1' [1]"Parsed with column specification:
cols(
  X1 = col_integer(),
  l = col_double(),
  `l*p` = col_double(),
  `l*p+r` = col_double(),
  `l*r` = col_double(),
  `l*r*p` = col_double(),
  `l*r+p` = col_double(),
  `l+p` = col_double(),
  `l+r` = col_double(),
  `l+r*p` = col_double(),
  `l+r+p` = col_double(),
  p = col_double(),
  `p*r` = col_double(),
  `p+r` = col_double(),
  r = col_double()
)
Parsed with column specification:
cols(
  cid = col_integer(),
  composition = col_character(),
  white_added = col_character(),
  second_exception = col_character()
)


In [239]:
metadata_prior <- metadata_prior %>%
                        mutate( cid_composition = paste0(cid, composition) )

metadata_prior$pid <- NULL
metadata_prior$composition <- NULL

kernels  <- c("l", "p", "r", "l+p", "l+r", "p+r", "l*r", "l*p", "p*r", "l+r+p", "l+r*p", "l*r+p", "l*p+r", "l*r*p")

lmls_prior<- lmls_prior %>% 
                gather(kernel, lml, kernels)

colnames(lmls_prior) = c('id', 'kernel', 'lml')

lmls_prior <- merge(x = lmls_prior, y = dict_prior, by = c("id", "id"), all.x = TRUE)

lmls_prior <- lmls_prior %>%
                        mutate( cid_composition = paste0(id, kernel) )

# Joining
lmls_prior <- merge(x = lmls_prior, y = metadata_prior, by = c("cid_composition", "cid_composition"), all.x = TRUE)
lmls_prior$cid_composition <- NULL

In [240]:
# Kernel composition name change
lmls_prior$kernel <- readable_kernel(lmls_prior$kernel)

In [241]:
# Removes the GPs that failed to be optimized in the second time. The lmls that failed to be optimized are also removed.
#Finaly, the lml_standard is calculated
lmls_prior_f <- lmls_prior %>%
                    filter(second_exception == 'False' & lml != -999999999) %>%
                    group_by(pid, scenario) %>%
                    mutate(lml_minus_min = lml - min(lml),
                           lml_standard = lml_minus_min / (max(lml_minus_min) - min (lml_minus_min)))

sds <- lmls_prior_f %>% 
            group_by(kernel, scenario) %>%
            summarize(lo_bound = mean(lml) - 5*sd(lml), up_bound = mean(lml) + 5*sd(lml))

sds$ks <- paste0(sds$kernel, sds$scenario)

lmls_prior_f$ks <- paste0(lmls_prior_f$kernel, lmls_prior_f$scenario)

sds$kernel <- NULL
sds$scenario <- NULL

lmls_prior_f <- merge(x = lmls_prior_f, y = sds, by = "ks", all.x = TRUE)

#### Prepare the plots

In [242]:
normalize_bool = TRUE
plot_bool      = TRUE

to_plot_prior <- lmls_prior_f %>% 
            filter(lml > -10000) %>%
            filter(lml > lo_bound & lml < up_bound) %>%
            group_by(kernel, scenario) %>%
            summarize(value = mean(lml),
                      lo_ci = MeanCI(lml, method="boot", type="norm", na.rm=TRUE)['lwr.ci'],
                      hi_ci = MeanCI(lml, method="boot", type="norm", na.rm=TRUE)['upr.ci'])

# Remove single components
to_plot_prior <- to_plot_prior %>%
                filter(kernel != 'l', kernel != 'p', kernel != 'r')

# Scaling the 'data to plot' to a 0-1 range
min_v = min(to_plot_prior$value)
range_v = max(to_plot_prior$value) - min(to_plot_prior$value)

if(normalize_bool) {
    #Normalization
    to_plot_prior <- to_plot_prior %>%
                    group_by( scenario ) %>%
                    mutate( lo_ci = (lo_ci - min(value)) / (max(value) - min(value)),
                            hi_ci = (hi_ci - min(value)) / (max(value) - min(value)),  
                            value = (value - min(value)) / (max(value) - min(value)))
}

# Plotting magic
to_plot_prior$kernel <- factor(to_plot_prior$kernel, levels=readable_kernel(kernels2))

lmls_temperature <- to_plot_prior %>% filter(scenario == "Temperature")
lmls_rain <- to_plot_prior %>% filter(scenario == "Rain")
lmls_sales <- to_plot_prior %>% filter(scenario == "Sales")
lmls_gym <- to_plot_prior %>% filter(scenario == "Gym members")
lmls_salary <- to_plot_prior %>% filter(scenario == "Salary")
lmls_fb <- to_plot_prior %>% filter(scenario == "FB Friends")


if (plot_bool) {
    # Plots
    p_temp   <- plot_lmls( lmls_temperature, scenario="Temperature", hide_x=TRUE, hide_y=TRUE)
    p_rain   <- plot_lmls( lmls_rain,        scenario="Rain",        hide_x=TRUE, hide_y=TRUE)
    p_sales  <- plot_lmls( lmls_sales,       scenario="Sales",       hide_x=TRUE, hide_y=TRUE)
    p_gym    <- plot_lmls( lmls_gym,         scenario="Gym members", hide_x=TRUE, hide_y=TRUE)
    p_salary <- plot_lmls( lmls_salary,      scenario="Salary",      hide_x=TRUE, hide_y=TRUE)
    p_fb     <- plot_lmls( lmls_fb,          scenario="FB Friends",  hide_x=TRUE, hide_y=TRUE)

    #pdf("Images/paper_images/kernels_priors_lmls_5sd.pdf", width=8, height=4)
    #multiplot(p1, p2, p3, p4, p5, p6, cols=3)
    #dev.off()   
}

Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)
Parsed with column specification:
cols(
  scenario = col_character(),
  kernel = col_character(),
  lml = col_double()
)


# Prior: participant's data

#### Plotting

In [243]:
mean_centering <- function(rwdata, dat, scenario) {
    
    mean_rw <- rwdata %>% filter(scenario == !!scenario) %>% summarize(mv=mean(value))
    
    mean_dat <- dat %>% filter(scenario == !!scenario, condition == 'Prior') %>% summarize(mv=mean(value))    
    
    mean_dat$mv / mean_rw$mv
}

# Real-world data
rwdata <- read_csv('data/real-world/splines.csv')

rwdata$condition <- 'Prior' # Add column
rwdata$id <- 1 #Create a new ID column
rwdata <- filter(rwdata, day >= 31) # Filter out the initial days
rwdata <- filter(rwdata, day <= 365*4 - 31) # Filter out the final days

# Order
rwdata$condition <- factor( rwdata$condition, levels = condition_names)
rwdata$scenario <- factor( rwdata$scenario, levels = readable_scenarios)

# Scale the values
rwdata[rwdata$scenario == 'Rain', 'day'] <- rwdata[rwdata$scenario == 'Rain', 'day'] + (365/12) * 1.5 #Offset 1.5 months to the right

# Mean centering
for (s in readable_scenarios){
    rwdata[rwdata$scenario == s, 'value'] <- rwdata[rwdata$scenario == s, 'value'] * mean_centering(rwdata, dat, s)
}

Parsed with column specification:
cols(
  scenario = col_character(),
  day = col_integer(),
  value = col_double()
)


In [244]:
curves_plot <- function(scenario_name, ...) {
    dat %>% 
        filter(scenario==scenario_name, condition=='Prior') %>% 
        plot_curves(rwdata %>% filter(scenario==scenario_name), ylab=scenario_name, ...)
}

In [245]:
p_temp_curves   <- curves_plot('Temperature', hide_x=TRUE, hide_y=FALSE)
p_rain_curves   <- curves_plot('Rain',        hide_x=TRUE, hide_y=FALSE)
p_sales_curves  <- curves_plot('Sales',       hide_x=TRUE, hide_y=FALSE)
p_gym_curves    <- curves_plot('Gym members', hide_x=TRUE, hide_y=FALSE)
p_salary_curves <- curves_plot('Salary',      hide_x=TRUE, hide_y=FALSE)
p_fb_curves     <- curves_plot('FB Friends',  hide_x=TRUE, hide_y=FALSE)

In [246]:
if(TRUE){
    # Save/show
    pdf("Images/paper_images/grid_plot_prior.pdf", width=6, height=10)
    grid.arrange(p_temp_curves,   p_temp, 
                 p_rain_curves,   p_rain, 
                 p_sales_curves,  p_sales, 
                 p_gym_curves,    p_gym, 
                 p_salary_curves, p_salary, 
                 p_fb_curves,     p_fb, 
                 ncol=2)
    dev.off()   
}

png 
  2